In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

In [4]:
print(char_dic)

{'e': 0, 'c': 1, 'p': 2, "'": 3, 'b': 4, 'r': 5, 'u': 6, 'o': 7, 'n': 8, 'f': 9, 'y': 10, '.': 11, 'm': 12, 'g': 13, 'd': 14, 's': 15, ',': 16, 'i': 17, 'a': 18, 'l': 19, 'h': 20, ' ': 21, 'w': 22, 't': 23, 'k': 24}


In [5]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [6]:
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

In [7]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i+1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
print(x_data[0])
print(y_data[0])

[17, 9, 21, 10, 7, 6, 21, 22, 18, 8]
[9, 21, 10, 7, 6, 21, 22, 18, 8, 23]


In [14]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [15]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [16]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [17]:
print(Y[0])

tensor([ 9, 21, 10,  7,  6, 21, 22, 18,  8, 23])


In [18]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [19]:
net = Net(dic_size, hidden_size, 2)

In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [21]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [22]:
print(outputs.view(-1, dic_size).shape)

torch.Size([1700, 25])


In [23]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [24]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

fr c cf fscfc fcc      f ff     fcf f f  f ff fff c fff ffff ff ff  cff fff   f   fcf cc   f  f ff c f    f  f ffccf f cffffff   fc f ff ff    f ffc f ff f      ff   crs m c f f  
                                                                                                                                                                                   
   tt  ot  e        t  t  t    t  e     t       t         t           t   tt       t      t      t          t   t        t n           t      nt  t    t   t   t    t      e       
,ffosfooskhwosytiwosfo,hs,dsfosfthwosfd,osfd,fdfdfhd,tfwos,osytpwd,tkwosfdwoi,fosfohwoskdfffoeo ,hsoskdfdfwoskfofoiwds,hers.osrtpfo,fs,osytsfdfo,yfoersfdffos,wfs kdffoskopfoioskof
eoeheswoosheosfoes.os e  ..o  emo sishos eoeo .woshes hes hoechosiosheosioeos worithoi iw   her .oe ihe s eorshosho a s ecshoecheo er .oepwortoosioorcwhoiworiwosi.o ihemeoshessho 
           e                      e                                                 e        e      

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
p you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
p you want to build a ship, don't drum up people together to collect wood and don't assign them task